In [1]:
import os
import sys
from collections import Counter

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

    
    
from enum import Enum

import cv2


class MethodAveragePrecision(Enum):
    """
    Class representing if the coordinates are relative to the
    image size or are absolute values.

        Developed by: Rafael Padilla
        Last modification: Apr 28 2018
    """
    EveryPointInterpolation = 1
    ElevenPointInterpolation = 2


class CoordinatesType(Enum):
    """
    Class representing if the coordinates are relative to the
    image size or are absolute values.

        Developed by: Rafael Padilla
        Last modification: Apr 28 2018
    """
    Relative = 1
    Absolute = 2


class BBType(Enum):
    """
    Class representing if the bounding box is groundtruth or not.

        Developed by: Rafael Padilla
        Last modification: May 24 2018
    """
    GroundTruth = 1
    Detected = 2


class BBFormat(Enum):
    """
    Class representing the format of a bounding box.
    It can be (X,Y,width,height) => XYWH
    or (X1,Y1,X2,Y2) => XYX2Y2

        Developed by: Rafael Padilla
        Last modification: May 24 2018
    """
    XYWH = 1
    XYX2Y2 = 2


# size => (width, height) of the image
# box => (X1, X2, Y1, Y2) of the bounding box
def convertToRelativeValues(size, box):
    dw = 1. / (size[0])
    dh = 1. / (size[1])
    cx = (box[1] + box[0]) / 2.0
    cy = (box[3] + box[2]) / 2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = cx * dw
    y = cy * dh
    w = w * dw
    h = h * dh
    # x,y => (bounding_box_center)/width_of_the_image
    # w => bounding_box_width / width_of_the_image
    # h => bounding_box_height / height_of_the_image
    return (x, y, w, h)


# size => (width, height) of the image
# box => (centerX, centerY, w, h) of the bounding box relative to the image
def convertToAbsoluteValues(size, box):
    # w_box = round(size[0] * box[2])
    # h_box = round(size[1] * box[3])
    xIn = round(((2 * float(box[0]) - float(box[2])) * size[0] / 2))
    yIn = round(((2 * float(box[1]) - float(box[3])) * size[1] / 2))
    xEnd = xIn + round(float(box[2]) * size[0])
    yEnd = yIn + round(float(box[3]) * size[1])
    if xIn < 0:
        xIn = 0
    if yIn < 0:
        yIn = 0
    if xEnd >= size[0]:
        xEnd = size[0] - 1
    if yEnd >= size[1]:
        yEnd = size[1] - 1
    return (xIn, yIn, xEnd, yEnd)


def add_bb_into_image(image, bb, color=(255, 0, 0), thickness=2, label=None):
    r = int(color[0])
    g = int(color[1])
    b = int(color[2])

    font = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 0.5
    fontThickness = 1

    x1, y1, x2, y2 = bb.getAbsoluteBoundingBox(BBFormat.XYX2Y2)
    x1 = int(x1)
    y1 = int(y1)
    x2 = int(x2)
    y2 = int(y2)
    cv2.rectangle(image, (x1, y1), (x2, y2), (b, g, r), thickness)
    # Add label
    if label is not None:
        # Get size of the text box
        (tw, th) = cv2.getTextSize(label, font, fontScale, fontThickness)[0]
        # Top-left coord of the textbox
        (xin_bb, yin_bb) = (x1 + thickness, y1 - th + int(12.5 * fontScale))
        # Checking position of the text top-left (outside or inside the bb)
        if yin_bb - th <= 0:  # if outside the image
            yin_bb = y1 + th  # put it inside the bb
        r_Xin = x1 - int(thickness / 2)
        r_Yin = y1 - th - int(thickness / 2)
        # Draw filled rectangle to put the text in it
        cv2.rectangle(image, (r_Xin, r_Yin - thickness),
                      (r_Xin + tw + thickness * 3, r_Yin + th + int(12.5 * fontScale)), (b, g, r),
                      -1)
        cv2.putText(image, label, (xin_bb, yin_bb), font, fontScale, (0, 0, 0), fontThickness,
                    cv2.LINE_AA)
    return image






class BoundingBoxes:
    def __init__(self):
        self._boundingBoxes = []

    def addBoundingBox(self, bb):
        self._boundingBoxes.append(bb)

    def removeBoundingBox(self, _boundingBox):
        for d in self._boundingBoxes:
            if BoundingBox.compare(d, _boundingBox):
                del self._boundingBoxes[d]
                return

    def removeAllBoundingBoxes(self):
        self._boundingBoxes = []

    def getBoundingBoxes(self):
        return self._boundingBoxes

    def getBoundingBoxByClass(self, classId):
        boundingBoxes = []
        for d in self._boundingBoxes:
            if d.getClassId() == classId:  # get only specified bounding box type
                boundingBoxes.append(d)
        return boundingBoxes

    def getClasses(self):
        classes = []
        for d in self._boundingBoxes:
            c = d.getClassId()
            if c not in classes:
                classes.append(c)
        return classes

    def getBoundingBoxesByType(self, bbType):
        # get only specified bb type
        return [d for d in self._boundingBoxes if d.getBBType() == bbType]

    def getBoundingBoxesByImageName(self, imageName):
        # get only specified bb type
        return [d for d in self._boundingBoxes if d.getImageName() == imageName]

    def count(self, bbType=None):
        if bbType is None:  # Return all bounding boxes
            return len(self._boundingBoxes)
        count = 0
        for d in self._boundingBoxes:
            if d.getBBType() == bbType:  # get only specified bb type
                count += 1
        return count

    def clone(self):
        newBoundingBoxes = BoundingBoxes()
        for d in self._boundingBoxes:
            det = BoundingBox.clone(d)
            newBoundingBoxes.addBoundingBox(det)
        return newBoundingBoxes

    def drawAllBoundingBoxes(self, image, imageName):
        bbxes = self.getBoundingBoxesByImageName(imageName)
        for bb in bbxes:
            if bb.getBBType() == BBType.GroundTruth:  # if ground truth
                image = add_bb_into_image(image, bb, color=(0, 255, 0))  # green
            else:  # if detection
                image = add_bb_into_image(image, bb, color=(255, 0, 0))  # red
        return image

    
    
from utils1 import *


class BoundingBox:
    def __init__(self,
                 imageName,
                 classId,
                 x,
                 y,
                 w,
                 h,
                 typeCoordinates=CoordinatesType.Absolute,
                 imgSize=None,
                 bbType=BBType.GroundTruth,
                 classConfidence=None,
                 format=BBFormat.XYX2Y2):
        self._imageName = imageName
        self._typeCoordinates = typeCoordinates
        if typeCoordinates == CoordinatesType.Relative and imgSize is None:
            raise IOError(
                'Parameter \'imgSize\' is required. It is necessary to inform the image size.')
        if bbType == BBType.Detected and classConfidence is None:
            raise IOError(
                'For bbType=\'Detection\', it is necessary to inform the classConfidence value.')
        # if classConfidence != None and (classConfidence < 0 or classConfidence > 1):
        # raise IOError('classConfidence value must be a real value between 0 and 1. Value: %f' %
        # classConfidence)

        self._classConfidence = classConfidence
        self._bbType = bbType
        self._classId = classId
        self._format = format

        # If relative coordinates, convert to absolute values
        # For relative coords: (x,y,w,h)=(X_center/img_width , Y_center/img_height)
        if (typeCoordinates == CoordinatesType.Relative):
            (self._x, self._y, self._w, self._h) = convertToAbsoluteValues(imgSize, (x, y, w, h))
            self._width_img = imgSize[0]
            self._height_img = imgSize[1]
            if format == BBFormat.XYWH:
                self._x2 = self._w
                self._y2 = self._h
                self._w = self._x2 - self._x
                self._h = self._y2 - self._y
            else:
                raise IOError(
                    'For relative coordinates, the format must be XYWH (x,y,width,height)')
        # For absolute coords: (x,y,w,h)=real bb coords
        else:
            self._x = x
            self._y = y
            if format == BBFormat.XYWH:
                self._w = w
                self._h = h
                self._x2 = self._x + self._w
                self._y2 = self._y + self._h
            else:  # format == BBFormat.XYX2Y2: <left> <top> <right> <bottom>.
                self._x2 = w
                self._y2 = h
                self._w = self._x2 - self._x
                self._h = self._y2 - self._y
        if imgSize is None:
            self._width_img = None
            self._height_img = None
        else:
            self._width_img = imgSize[0]
            self._height_img = imgSize[1]

    def getAbsoluteBoundingBox(self, format=BBFormat.XYWH):
        if format == BBFormat.XYWH:
            return (self._x, self._y, self._w, self._h)
        elif format == BBFormat.XYX2Y2:
            return (self._x, self._y, self._x2, self._y2)

    def getRelativeBoundingBox(self, imgSize=None):
        if imgSize is None and self._width_img is None and self._height_img is None:
            raise IOError(
                'Parameter \'imgSize\' is required. It is necessary to inform the image size.')
        if imgSize is not None:
            return convertToRelativeValues((imgSize[0], imgSize[1]),
                                           (self._x, self._x2, self._y, self._y2))
        else:
            return convertToRelativeValues((self._width_img, self._height_img),
                                           (self._x, self._x2, self._y, self._y2))

    def getImageName(self):
        return self._imageName

    def getConfidence(self):
        return self._classConfidence

    def getFormat(self):
        return self._format

    def getClassId(self):
        return self._classId

    def getImageSize(self):
        return (self._width_img, self._height_img)

    def getCoordinatesType(self):
        return self._typeCoordinates

    def getBBType(self):
        return self._bbType

    @staticmethod
    def compare(det1, det2):
        det1BB = det1.getAbsoluteBoundingBox()
        det1ImgSize = det1.getImageSize()
        det2BB = det2.getAbsoluteBoundingBox()
        det2ImgSize = det2.getImageSize()

        if det1.getClassId() == det2.getClassId() and \
           det1.classConfidence == det2.classConfidenc() and \
           det1BB[0] == det2BB[0] and \
           det1BB[1] == det2BB[1] and \
           det1BB[2] == det2BB[2] and \
           det1BB[3] == det2BB[3] and \
           det1ImgSize[0] == det2ImgSize[0] and \
           det1ImgSize[1] == det2ImgSize[1]:
            return True
        return False

    @staticmethod
    def clone(boundingBox):
        absBB = boundingBox.getAbsoluteBoundingBox(format=BBFormat.XYWH)
        # return (self._x,self._y,self._x2,self._y2)
        newBoundingBox = BoundingBox(boundingBox.getImageName(),
                                     boundingBox.getClassId(),
                                     absBB[0],
                                     absBB[1],
                                     absBB[2],
                                     absBB[3],
                                     typeCoordinates=boundingBox.getCoordinatesType(),
                                     imgSize=boundingBox.getImageSize(),
                                     bbType=boundingBox.getBBType(),
                                     classConfidence=boundingBox.getConfidence(),
                                     format=BBFormat.XYWH)
        return newBoundingBox


import argparse
#from keras_yolo3.yolo import YOLO, detect_video, detect_webcam
from PIL import Image
from timeit import default_timer as timer
#from utils1 import load_extractor_model, load_features, parse_input, detect_object
import test
#import utils
import pandas as pd
import numpy as np
#from Get_File_Paths import GetFileList
import random
#from Train_Utils import get_anchors

#gt_csv = pd.read_csv("Detection_Results.csv")
#print(type(gt_csv))
#pred_csv = pd.read_csv("Detection_Results.csv")
#print(type(pred_csv))
#IOUThreshold=0.80
def getBoundingBoxes(confidence):
    allBoundingBoxes = BoundingBoxes()
    for i in range(len(gt_csv)):
        nameOfImage = gt_csv['image'][i]
        idClass = gt_csv['label'][i]
        x = float(gt_csv['xmin'][i])
        y = float(gt_csv['ymin'][i])
        w = float(gt_csv['xmax'][i])
        h = float(gt_csv['ymax'][i])
        bb = BoundingBox(
        nameOfImage,
        idClass,
        x,
        y,
        w,
        h,
        CoordinatesType.Absolute, (200, 200),
        BBType.GroundTruth,
        format=BBFormat.XYX2Y2)
        allBoundingBoxes.addBoundingBox(bb)

    for i in range(len(pred_csv)):
        nameOfImage = pred_csv['image'][i]
        idClass = pred_csv['label'][i]
        #confidence = float(pred_csv['confidence'][i])#.astype(float)
        x = float(pred_csv['xmin'][i])#.astype(float)
        y = float(pred_csv['ymin'][i])#.astype(float)
        w = float(pred_csv['xmax'][i])#.astype(float)
        h = float(pred_csv['ymax'][i])#.astype(float) #.apply(lambda x : float(x))
        bb = BoundingBox(
        nameOfImage,
        idClass,
        x,
        y,
        w,
        h,
        CoordinatesType.Absolute, (200, 200),
        BBType.Detected,confidence,
        format=BBFormat.XYX2Y2)
        allBoundingBoxes.addBoundingBox(bb)

    return allBoundingBoxes

##############################################################################################################################


class Evaluator:
    def GetPascalVOCMetrics(self,
                            boundingboxes,
                            IOUThreshold=0.20,
                            method=MethodAveragePrecision.EveryPointInterpolation):
        
        ret = []  # list containing metrics (precision, recall, average precision) of each class
        # List with all ground truths (Ex: [imageName,class,confidence=1, (bb coordinates XYX2Y2)])
        groundTruths = []
        # List with all detections (Ex: [imageName,class,confidence,(bb coordinates XYX2Y2)])
        detections = []
        # Get all classes
        classes = []
        # Loop through all bounding boxes and separate them into GTs and detections
        for bb in boundingboxes.getBoundingBoxes():
            # [imageName, class, confidence, (bb coordinates XYX2Y2)]
            if bb.getBBType() == BBType.GroundTruth:
                groundTruths.append([
                    bb.getImageName(),
                    bb.getClassId(), 1,
                    bb.getAbsoluteBoundingBox(BBFormat.XYX2Y2)
                ])
            else:
                detections.append([
                    bb.getImageName(),
                    bb.getClassId(),
                    bb.getConfidence(),
                    bb.getAbsoluteBoundingBox(BBFormat.XYX2Y2)
                ])
            # get class
            if bb.getClassId() not in classes:
                classes.append(bb.getClassId())
        classes = sorted(classes)
        # Precision x Recall is obtained individually by each class
        # Loop through by classes
        count=0
        images_fp=[]
        images_tp=[]
        images_fn=[]
        fn_dim=[]
        fp_dim=[]
        tp_dim=[]
        for c in classes:
            # Get only detection of class c
            dects = []
            [dects.append(d) for d in detections if d[1] == c]
            # Get only ground truths of class c, use filename as key
            gts = {}
            npos = 0
            for g in groundTruths:
                if g[1] == c:
                    npos += 1
                    gts[g[0]] = gts.get(g[0], []) + [g]

            #print(groundTruths)
            #exit(0)
            # sort detections by decreasing confidence
            dects = sorted(dects, key=lambda conf: conf[2], reverse=True)
            TP = np.zeros(len(dects))
            FP = np.zeros(len(dects))
            # create dictionary with amount of gts for each image
            det = {key: np.zeros(len(gts[key])) for key in gts}

            # print("Evaluating class: %s (%d detections)" % (str(c), len(dects)))
            # Loop through groundtruths one by one
            acc_FP = 0
            acc_TP = 0
            acc_FN = 0
            p_tp=[]
            p_fp=[]
            p_fn=[]
                    
            for g in groundTruths:

                iouMax = sys.float_info.min
                iou = 0.0
                found = False
                p=[]
                for d in range(len(dects)):
                    # print('Ground truth gt => %s' % (gt[j][3],))
                    if dects[d][0] == g[0]:
                        found = True
                        #print("found=true",dects[d][0], g[0])
                        #if (Evaluator.iou(dects[d][3], gts[key][0][3]) == 0):
                            #for case 6 in document
                        #    acc_FP = acc_FP + 1
                        iou = iou + Evaluator.iou(dects[d][3], g[3])
                        p.append(dects[d])
                    #else:
                        #p_fp.append(dects[d])
                        #print(p_fp,"##################")
                        
                        #print("iou", iou, "d[3]", dects[d][3],"g[3]", g[3])
                #No Detections for the groundtruth
                
            
                #print(len(p))
                if found == False:
                    ###FALSE POSITIVE(IT IS ACTUALLY NOT THERE IN GT BUT EXISTS IN DETECTION)
                    #print("it is FP")
                    acc_FP = acc_FP + 1  #acc_FP gives count of False positives
                    #print("acc_FP", len(acc_FP))
                    #images_fp.append(dects[d][0])
                    #fp_dim.append(d)   #fp_dim saves the dimensions of false positives
                    #print(images_fp, "****************************************")
                    continue
                # Assign detection as true positive/don't care/false positive
                if iou >= IOUThreshold:
                    acc_TP = acc_TP + 1  # count as true positive
                    #images_tp.append(dects[d][0])
                    tp_dim.append(g)
                    p_tp.append(p)
                    
                else:
                    acc_FN = acc_FN + 1  # count of fasle negatives
                    #images_fn.append(dects[d][0])
                    fn_dim.append(g) 
                    print(fn_dim)
                    p_fn.append(p)
                    #print(images_fn)
                    # print("FP")
            # compute precision, recall and average precision
            #exit()
            #print("images_fn", images_fn)
            #print(len(images_fn))
            #print("images_tp", images_tp)
            #print("images_fp", images_fp)
            #print("dimensions of fn", len(fn_dim), fn_dim[3])
            #print("dimensions of tp", len(tp_dim), tp_dim[3])
            #print("dimensions of fp", len(fp_dim), fp_dim[0])
            #print('\n\nTotal Positives: %s' % npos)
            #print('\n\nTrue  Positives: %s' % acc_TP)
            #print('\n\nFalse Positives: %s' % acc_FP)
            #print('\n\nFalse Negatives: %s' % acc_FN)

            rec = acc_TP / npos
            prec = np.divide(acc_TP, (acc_FP + acc_TP))

            count = count+1
            #print(count)
            r = {
                'class': c,
                'precision': prec,
                'recall': rec,
                'AP': 0.0,
                'interpolated precision': 0.0,
                'interpolated recall': 0.0,
                'total positives': npos,
                'total TP': acc_TP,
                'total FP': acc_FP,
                'total FN': npos-acc_TP
            }
            ret.append(r)
            #print("====\n============================",fn_dim)
            
            ###########################################################################################################
            print("Object Detection Metrics for Wind Turbines Model")
            print("confidence:", confidence)
            print("IOU Threshold:", IOUThreshold)
            print("Number of TP:", acc_TP)
            print("Number of FP:", acc_FP)
            print("Number of FN:", acc_FN)
            Recall_n = acc_TP/(acc_TP+acc_FN)
            Precision_n = acc_TP/(acc_TP+acc_FP)
            F_measure_n = (2*Precision_n*Recall_n)/(Precision_n+Recall_n)            
            
            print("Recall:", Recall_n)
            print("Precision", Precision_n)
            print("F-Measure", F_measure_n)
            print("TOTal GT",len(groundTruths))
            

            '''df_fp=pd.DataFrame(p_fp)
            #df_fp.columns=['image', 'class','confidence','BB_dim']
            #print(df_fp)
            #df_fp.drop(df_fp.iloc[:, 4:6], inplace=True, axis=1)
            #print(df_fp)
            ##########################################################

            
            #Dimensions of FP
            temp = []
            detect_list = pred_csv.values.tolist()
            GTT = gt_csv['image'].tolist()
            for i in range(len(detect_list)):
                count=0
                for j in range(len(GTT)):
                    if GTT[j]==detect_list[i][0]:
                        count+=1
                if count<1:
                    temp.append(detect_list[i])
            print(len(temp))
            fp_dimensions = pd.DataFrame(temp)
            fp_dimensions.columns=['image','xmin','ymin','xmax','ymax','label','confidence']
            fp_dimensions.to_csv('FP_Dimensions.csv')'''
                    
                        
                    
            '''#To print only the dimensions of FPs which are in sync with detection results.
            #print(pred_csv['image'])
            #print(df_fp_image_names['image'])
            #df_new = pred_csv[pred_csv['image'] == df_fp_image_names['image']]
            #print(df_new)
            dimension_fp=[]
            temp = pred_csv['image'].tolist()
            #print(temp,"+++####")
            #a = temp[0]
            #print(a)
            #b = fp_dim[0][0]
            #print(b)
            #c = len(a)
            #print(c)
            for i in temp:
                #print(i)
                for j in range(len(fp_dim)):
                    if fp_dim[j][0].startswith(i):
                        print(fp_dim[j][0])
                    #print((i))
                    #print(type(fp_dim[j][0]))
                    if (fp_dim[j][0].strip()==i.strip()):
                        print("##",fp_dim[j][0],"##")
                        dimension_fp.append(fp_dim[j][:])
                        print(dimension_fp)'''
                        
                        
            #############################################################
            #df_tp=pd.DataFrame(p_tp)
            #print(df_tp)
            #df_tp.columns=['image', 'class','confidence','BB_dim']
            #############################################################
            ### For TP images into CSV
            
            df_tp_image_names=pd.DataFrame(tp_dim)
            #df_tp_image_names.columns=['image', 'class','confidence','BB_dim']
            print(df_tp_image_names)
            df_tp_image_names.to_csv('TP_images.csv')
            ###############################################################
            df_fn=pd.DataFrame(p_fn)
            print(df_fn)
            #df_fn.columns=['image', 'class','confidence','BB_dim']
            #####################################################################
            ### For FN images into CSV
            
            df_fn_image_names=pd.DataFrame(fn_dim)
            df_fn_image_names.columns=['image', 'class','confidence','BB_dim']
            df_fn_image_names.to_csv('FN_images.csv')
            print("written to FN")
            #print(df_fp.info(),df_fp_image_names.info(),df_tp.info(),df_tp_image_names.info(),df_fn.info(),df_fn_image_names.info())
            #########################################################################
            #for FP images into csv
                        
            ######For FP images in csv
            df_fp_image_names=pd.DataFrame(fp_dim)
            #df_fp_image_names.columns=['image', 'label','confidence', 'BB_dim']
            print("*****",df_fp_image_names)
            df_fp_image_names.to_csv('FP_images.csv')
            
            
            
            #######################################################################
        return ret

    def PlotPrecisionRecallCurve(self,
                                 boundingBoxes,
                                 IOUThreshold=0.20,
                                 method=MethodAveragePrecision.EveryPointInterpolation,
                                 showAP=False,
                                 showInterpolatedPrecision=False,
                                 savePath=None,
                                 showGraphic=True):

        results = self.GetPascalVOCMetrics(boundingBoxes, IOUThreshold, method)
        result = None
        # Each resut represents a class
        for result in results:
            if result is None:
                raise IOError('Error: Class %d could not be found.' % classId)

            classId = result['class']
            precision = result['precision']
            recall = result['recall']
            average_precision = result['AP']
            mpre = result['interpolated precision']
            mrec = result['interpolated recall']
            npos = result['total positives']
            total_tp = result['total TP']
            total_fp = result['total FP']
            total_fn = result['total FN']

            plt.close()
            if showInterpolatedPrecision:
                if method == MethodAveragePrecision.EveryPointInterpolation:
                    plt.plot(mrec, mpre, '--r', label='Interpolated precision (every point)')
                elif method == MethodAveragePrecision.ElevenPointInterpolation:
                    # Uncomment the line below if you want to plot the area
                    # plt.plot(mrec, mpre, 'or', label='11-point interpolated precision')
                    # Remove duplicates, getting only the highest precision of each recall value
                    nrec = []
                    nprec = []
                    for idx in range(len(mrec)):
                        r = mrec[idx]
                        if r not in nrec:
                            idxEq = np.argwhere(mrec == r)
                            nrec.append(r)
                            nprec.append(max([mpre[int(id)] for id in idxEq]))
                    plt.plot(nrec, nprec, 'or', label='11-point interpolated precision')
            plt.plot(recall, precision, label='Precision')
            plt.xlabel('recall')
            plt.ylabel('precision')
            if showAP:
                ap_str = "{0:.2f}%".format(average_precision * 100)
                # ap_str = "{0:.4f}%".format(average_precision * 100)
                plt.title('Precision x Recall curve \nClass: %s, AP: %s' % (str(classId), ap_str))
            else:
                plt.title('Precision x Recall curve \nClass: %s' % str(classId))
            plt.legend(shadow=True)
            plt.grid()
            if savePath is not None:
                plt.savefig(os.path.join(savePath, str(classId) + '.png'))
            if showGraphic is True:
                plt.show()
                # plt.waitforbuttonpress()
                plt.pause(0.05)
        return results

    @staticmethod
    def CalculateAveragePrecision(rec, prec):
        mrec = []
        mrec.append(0)
        [mrec.append(e) for e in rec]
        mrec.append(1)
        mpre = []
        mpre.append(0)
        [mpre.append(e) for e in prec]
        mpre.append(0)
        for i in range(len(mpre) - 1, 0, -1):
            mpre[i - 1] = max(mpre[i - 1], mpre[i])
        ii = []
        for i in range(len(mrec) - 1):
            if mrec[1+i] != mrec[i]:
                ii.append(i + 1)
        ap = 0
        for i in ii:
            ap = ap + np.sum((mrec[i] - mrec[i - 1]) * mpre[i])
        # return [ap, mpre[1:len(mpre)-1], mrec[1:len(mpre)-1], ii]
        return [ap, mpre[0:len(mpre) - 1], mrec[0:len(mpre) - 1], ii]

    @staticmethod
    # 11-point interpolated average precision
    def ElevenPointInterpolatedAP(rec, prec):
        # def CalculateAveragePrecision2(rec, prec):
        mrec = []
        # mrec.append(0)
        [mrec.append(e) for e in rec]
        # mrec.append(1)
        mpre = []
        # mpre.append(0)
        [mpre.append(e) for e in prec]
        # mpre.append(0)
        recallValues = np.linspace(0, 1, 11)
        recallValues = list(recallValues[::-1])
        rhoInterp = []
        recallValid = []
        # For each recallValues (0, 0.1, 0.2, ... , 1)
        for r in recallValues:
            # Obtain all recall values higher or equal than r
            argGreaterRecalls = np.argwhere(mrec[:] >= r)
            pmax = 0
            # If there are recalls above r
            if argGreaterRecalls.size != 0:
                pmax = max(mpre[argGreaterRecalls.min():])
            recallValid.append(r)
            rhoInterp.append(pmax)
        # By definition AP = sum(max(precision whose recall is above r))/11
        ap = sum(rhoInterp) / 11
        # Generating values for the plot
        rvals = []
        rvals.append(recallValid[0])
        [rvals.append(e) for e in recallValid]
        rvals.append(0)
        pvals = []
        pvals.append(0)
        [pvals.append(e) for e in rhoInterp]
        pvals.append(0)
        # rhoInterp = rhoInterp[::-1]
        cc = []
        for i in range(len(rvals)):
            p = (rvals[i], pvals[i - 1])
            if p not in cc:
                cc.append(p)
            p = (rvals[i], pvals[i])
            if p not in cc:
                cc.append(p)
        recallValues = [i[0] for i in cc]
        rhoInterp = [i[1] for i in cc]
        return [ap, rhoInterp, recallValues, None]

    # For each detections, calculate IOU with reference
    @staticmethod
    def _getAllIOUs(reference, detections):
        ret = []
        bbReference = reference.getAbsoluteBoundingBox(BBFormat.XYX2Y2)
        # img = np.zeros((200,200,3), np.uint8)
        for d in detections:
            bb = d.getAbsoluteBoundingBox(BBFormat.XYX2Y2)
            iou = Evaluator.iou(bbReference, bb)
            # Show blank image with the bounding boxes
            # img = add_bb_into_image(img, d, color=(255,0,0), thickness=2, label=None)
            # img = add_bb_into_image(img, reference, color=(0,255,0), thickness=2, label=None)
            ret.append((iou, reference, d))  # iou, reference, detection
        # cv2.imshow("comparing",img)
        # cv2.waitKey(0)
        # cv2.destroyWindow("comparing")
        return sorted(ret, key=lambda i: i[0], reverse=True)  # sort by iou (from highest to lowest)

    @staticmethod
    def iou(boxA, boxB):
        # if boxes dont intersect
        if Evaluator._boxesIntersect(boxA, boxB) is False:
            return 0
        interArea = Evaluator._getIntersectionArea(boxA, boxB)
        #union = Evaluator._getUnionAreas(boxA, boxB, interArea=interArea)
        union = Evaluator._getArea(boxB)
        # intersection over union
        #print(interArea, "interArea")
        #print("union", union)
        iou = interArea / union
        #assert iou >= 0
        return iou

    # boxA = (Ax1,Ay1,Ax2,Ay2)
    # boxB = (Bx1,By1,Bx2,By2)
    @staticmethod
    def _boxesIntersect(boxA, boxB):
        if boxA[0] > boxB[2]:
            return False  # boxA is right of boxB
        if boxB[0] > boxA[2]:
            return False  # boxA is left of boxB
        if boxA[3] < boxB[1]:
            return False  # boxA is above boxB
        if boxA[1] > boxB[3]:
            return False  # boxA is below boxB
        return True

    @staticmethod
    def _getIntersectionArea(boxA, boxB):
        xA = max(boxA[0], boxB[0])
        yA = max(boxA[1], boxB[1])
        xB = min(boxA[2], boxB[2])
        yB = min(boxA[3], boxB[3])
        # intersection area
        return (xB - xA + 1) * (yB - yA + 1)

    @staticmethod
    def _getUnionAreas(boxA, boxB, interArea=None):
        area_A = Evaluator._getArea(boxA)
        area_B = Evaluator._getArea(boxB)
        if interArea is None:
            interArea = Evaluator._getIntersectionArea(boxA, boxB)
        return float(area_A + area_B - interArea)

    @staticmethod
    def _getArea(box):
        return (box[2] - box[0] + 1) * (box[3] - box[1] + 1)








def test_function(gt_csv, pred_csv, confidence):
    pred_csv['image'] = pred_csv['image'].replace('.jpg','')
    gt_csv['image'] = gt_csv['image'].replace('.jpg','')

    gt_csv['label'] = 0

    gt_csv = gt_csv.sort_values(by = 'image')
    pred_csv = pred_csv.sort_values(by = 'image')

    #print("========================================================")
    #print("Ground Truth annotations")
    #print(gt_csv)
    #print("========================================================")
    #print("Detected Annotations")
    #print(pred_csv)
    boundingboxes = getBoundingBoxes(confidence)
    evaluator = Evaluator()


    metricsPerClass = evaluator.GetPascalVOCMetrics(
    boundingboxes,  # Object containing all bounding boxes (ground truths and detections)
    #IOUThreshold=0.80,  # IOU threshold
    method=MethodAveragePrecision.EveryPointInterpolation)  # As the official matlab code
    print("Average precision values per class:\n")



    #acc_AP = 0
    #validClasses = 0
    print('Object Detection Metrics\n')
    #print('https://github.com/rafaelpadilla/Object-Detection-Metrics\n\n\n')
    #print('Average Precision (AP), Precision and Recall per class:')

    # each detection is a class
    #for metricsPerClass in detections:

    # Get metric values per each class
    #    cl = metricsPerClass['class']
    #    ap = metricsPerClass['AP']
    #    precision = metricsPerClass['precision']
    #    recall = metricsPerClass['recall']
    #    totalPositives = metricsPerClass['total positives']
    #    total_TP = metricsPerClass['total TP']
    #    total_FP = metricsPerClass['total FP']
    #    total_FN = metricsPerClass['total FN']
    #    print('\n\nTotal Positives: %s' % totalPositives)
    #    print('\n\nTotal True positive: %s' % total_TP)
    #    print('\n\nTotal False Positive: %s' % total_FP)
    #    print('\n\nTotal False Negative: %s' % total_FN)




    #    if totalPositives > 0:
    #        validClasses = validClasses + 1
    #        acc_AP = acc_AP + ap
    #        prec = ['%.2f' % p for p in precision]
    #        rec = ['%.2f' % r for r in recall]
    #        ap_str = "{0:.2f}%".format(ap * 100)
            # ap_str = "{0:.4f}%".format(ap * 100)
    #        print('AP: %s (%s)' % (ap_str, cl))
    #        print('\n\nClass: %s' % cl)
    #        print('\nAP: %s' % ap_str)
    #        print('\nPrecision: %s' % prec)
    #        print('\nRecall: %s' % rec)

    #mAP = acc_AP / validClasses
    #mAP_str = "{0:.2f}%".format(mAP * 100)
    #print('mAP: %s' % mAP_str)
    #f.write('\n\n\nmAP: %s' % mAP_str)

gt_csv = pd.read_csv("new_csv.csv")
#print(len(gt_csv),"gt")
pred_csv = pd.read_csv("new_turbines_Detection_Results.csv")
#print(len(pred_csv),"pred")
#IOUThreshold=0.25
confidence=0.25
test_function(gt_csv, pred_csv,confidence)



[['img-0506.jpg', 0, 1, (1616.0, 834.0, 1684.0, 963.0)]]
[['img-0506.jpg', 0, 1, (1616.0, 834.0, 1684.0, 963.0)], ['img-4499.jpg', 0, 1, (1099.0, 1190.0, 1185.0, 1264.0)]]
[['img-0506.jpg', 0, 1, (1616.0, 834.0, 1684.0, 963.0)], ['img-4499.jpg', 0, 1, (1099.0, 1190.0, 1185.0, 1264.0)], ['img-6022.jpg', 0, 1, (1770.0, 1334.0, 1871.0, 1515.0)]]
[['img-0506.jpg', 0, 1, (1616.0, 834.0, 1684.0, 963.0)], ['img-4499.jpg', 0, 1, (1099.0, 1190.0, 1185.0, 1264.0)], ['img-6022.jpg', 0, 1, (1770.0, 1334.0, 1871.0, 1515.0)], ['img-7900.jpg', 0, 1, (1026.0, 1654.0, 1155.0, 1942.0)]]
Object Detection Metrics for Wind Turbines Model
confidence: 0.25
IOU Threshold: 0.2
Number of TP: 204
Number of FP: 24
Number of FN: 4
Recall: 0.9807692307692307
Precision 0.8947368421052632
F-Measure 0.9357798165137614
TOTal GT 232
                              0  1  2                                 3
0                  img-0475.jpg  0  1   (1766.0, 942.0, 1817.0, 1084.0)
1                  img-0476.jpg  0  1  (1781.0

In [63]:
##Reading the GT and Detections into a list

df_1=pd.read_csv("new_csv.csv")
gt_list=df_1['image'].tolist()
#print(gt_list)
df_2=pd.read_csv("new_turbines_Detection_Results.csv")
dt_list=df_2['image'].tolist()
#print(dt_list)
missing=(set(dt_list).difference(gt_list))
print(len(missing))

80


In [51]:
print(gt_list)

['img-0475.jpg', 'img-0476.jpg', 'img-0477.jpg', 'img-0478.jpg', 'img-0491.jpg', 'img-0492.jpg', 'img-0493.jpg', 'img-0504.jpg', 'img-0505.jpg', 'img-0506.jpg', 'img-0507.jpg', 'img-0508.jpg', 'img-0539.jpg', 'img-0538.jpg', 'img-0537.jpg', 'img-0536.jpg', 'img-0535.jpg', 'img-0534.jpg', 'img-0533.jpg', 'img-0532.jpg', 'img-0544.jpg', 'img-0545.jpg', 'img-0569.jpg', 'img-0568.jpg', 'img-0567.jpg', 'img-0550.jpg', 'img-0549.jpg', 'img-0548.jpg', 'img-0547.jpg', 'img-0546.jpg', 'img-1426.jpg', 'img-1427.jpg', 'img-1428.jpg', 'img-1429.jpg', 'img-1461.jpg', 'img-1462.jpg', 'img-1463.jpg', 'img-1464.jpg', 'img-1465.jpg', 'img-1466.jpg', 'img-1481.jpg', 'img-1482.jpg', 'img-1483.jpg', 'img-1484.jpg', 'img-1485.jpg', 'img-1486.jpg', 'img-1487.jpg', 'img-1818.jpg', 'img-1819.jpg', 'img-2311.jpg', 'img-2312.jpg', 'img-2313.jpg', 'img-2314.jpg', 'img-2906.jpg', 'img-2907.jpg', 'img-3586.jpg', 'img-3586.jpg', 'img-3587.jpg', 'img-3587.jpg', 'img-3588.jpg', 'img-3588.jpg', 'img-3589.jpg', 'img-35

In [66]:
#Finding the images that are not in GT but are in Detections
missing=list(missing)
count=0
for i in range(len(missing)):
    if missing[i] in dt_list and missing[i] not in gt_list:
        print("done")
        count=count+1
    else:
        print("recheck")
print(count-len(missing))        

done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
0


In [55]:
#Writing those images to a file
with open('missing_in_gt.txt', 'w+') as f:
     
    # write elements of list
    for items in missing:
        f.write('%s\n' %items)
     
    print("File written successfully")
 
 
# close the file
f.close()

File written successfully


In [67]:
#adding _detected to a name of a image
new_missing=[]
for i in range(len(missing)):
    new_missing.append("_detected.".join(missing[i].split(".")))
print(len(new_missing))

80


In [68]:
#Writing to a file after adding _detected to the image file name

with open('new_missing_in_gt.txt', 'w+') as f:
     
    # write elements of list
    for items in new_missing:
        f.write('%s\n' %items)
     
    print("File written successfully")
 
 
# close the file
f.close()

File written successfully
